## Description

##### Implementation of "Deep Residual Learning for Image Recognition" paper - https://ieeexplore.ieee.org/document/7780459

### Libraries

In [1]:
import tensorflow as tf
import numpy as np
import typing

In [2]:
from ipynb.fs.full.residual_learning_layers import *

### Input

##### Let's assume we have an input image of [batch_size, image_width, image_height, features]

In [3]:
input_shape = (1, 500, 500, 3) # images of batch 100, size of 500x500 and rgb hene feaures of size 3
x = tf.random.normal(input_shape)
x = tf.constant(x, dtype=tf.float32)

In [4]:
x.shape

<tf.Tensor: shape=(1, 500, 500, 3), dtype=float32, numpy=
array([[[[-5.86231768e-01,  1.60114622e+00,  3.02300751e-01],
         [-8.86706829e-01, -1.73417282e+00, -1.13709009e+00],
         [-1.43502343e+00,  1.25142813e+00,  1.09249723e+00],
         ...,
         [ 7.21697450e-01, -5.98199189e-01, -5.71431369e-02],
         [-4.05235738e-01, -8.72637689e-01, -1.52320933e+00],
         [-1.02649629e+00, -1.93687975e-01, -1.95227587e+00]],

        [[ 7.42623687e-01, -1.24532831e+00,  1.24146223e-01],
         [ 8.00725341e-01,  4.93192345e-01,  8.54823530e-01],
         [ 6.10289395e-01, -1.27179694e+00, -1.79805785e-01],
         ...,
         [-1.38284490e-01,  5.69418728e-01, -9.31997538e-01],
         [ 9.48348939e-01,  8.24724913e-01,  2.15256882e+00],
         [-9.56536055e-01,  1.31456763e-01,  1.01385765e-01]],

        [[-5.32221198e-01, -1.22341490e+00, -9.49513435e-01],
         [-1.23176491e+00,  7.33573139e-01, -2.02876449e+00],
         [-1.84546041e+00, -2.58735418e+00

### Residual Block

In [5]:
class ResidualBlock(tf.Module):
    
    def __init__(self, 
                out_channels: int, # aka nr. filters
                kernel: int, # the h*w of filter, can be a tuple but for now let's set it to a sqaure
                stride_1: int, # the h_steps*w_steps of filter as it moves along input, set to a square for now
                stride_2: int,
                name = None):
        super(ResidualBlock, self).__init__(name)
        
        self.conv2d_layer_1 = Conv2D(out_channels=out_channels,
                                     kernel=kernel,
                                     stride=stride_1)
        self.conv2d_layer_2 = Conv2D(out_channels=out_channels,
                                     kernel=kernel,
                                     stride=stride_2)
        self.batch_normalization = tf.keras.layers.BatchNormalization()
        
    def __call__(self, x_in: tf.Tensor):
        
        x = self.conv2d_layer_1(x_in)
        x = self.batch_normalization(x)
        x = tf.nn.leaky_relu(x)
        x = self.conv2d_layer_2(x)
        x = self.batch_normalization(x)
        x = tf.nn.leaky_relu(x)
        
        if x.shape != x_in.shape: # input should be projected (using 1x1 conv2d layer with stride of 2) to match dimensions of F(x_in)
            conv2d_proj_layer = Conv2D(out_channels=x.shape[-1], kernel=1, stride=2)
            x_in = conv2d_proj_layer(x_in)
    
        x = tf.add(x, x_in) # for residual effect, add input to output for final output
        
        return x

### Test Residual Block

In [6]:
# setup prior layers to first residual block
conv2d_layer = Conv2D(out_channels=64, kernel=7, stride=2)
x_1 = conv2d_layer(x)

max_pool = tf.keras.layers.MaxPool2D(pool_size=2)
x_2 = max_pool(x_1)

In [7]:
# test first residual block
residual_block = ResidualBlock(out_channels=64, 
                               kernel=3, 
                               stride_1=1, 
                               stride_2=1)
residual_block_output = residual_block(x_2)
residual_block_output.shape

TensorShape([1, 125, 125, 64])

In [10]:
# test propagation of first residual block to second (testing dimensionality matching)
residual_block = ResidualBlock(out_channels=128, 
                               kernel=3, 
                               stride_1=2, 
                               stride_2=1)
residual_block_output_2 = residual_block(residual_block_output)
residual_block_output_2.shape

TensorShape([1, 63, 63, 128])